In [116]:
import astropy.units as u
import time
from numpy import *
import numpy as np

In [128]:
class archives():
     
    def __init__(self, ID):
        
        if not isinstance(ID, str):
            TypeError('ID must be a string which is resolvable by Simbad')
            return None
        
        self.input_ID = ID
        self.KIC = {'ID': None, 'entry': None}
        self.TIC = {'ID': None, 'entry': None}
        self.GDR2 = {'ID': None, 'entry': None}

    def __call__():
        self.query_simbad()
        self.query_KIC()
        self.query_TIC()
        self.query_gaia()
        
    def query_simbad(self):
        from astroquery.simbad import Simbad
        Simbad.add_votable_fields('sptype', 'ids')
        time.sleep(0.1)        
        self.simbad = {'ID': None, 'entry': None}
        
        try:
            self.simbad['entry'] = Simbad.query_object(self.input_ID)
        except Exception as ex:
            message = "An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args)
            print(message)
                    
        if not self.simbad['entry']:
            print(f'Unable to find {self.input_ID} with Simbad')        
        else:
            IDS = self.simbad['entry']['IDS']
            self.simbad['ID'] = str(IDS[0]).strip("b'").split('|') # should check for length
            
            prefixes = ['Gaia DR2', 'KIC']
            
            dicts = [self.GDR2, self.KIC]
            for i, prefix in enumerate(prefixes):
                tempid = [id for id in self.simbad['ID'] if prefix in id]
                if len(tempid) == 0:
                    print(f'Unable to resolve {prefix} ID for {self.input_ID} with Simbad')
                elif len(tempid) > 1:
                    print(f'Warning: for some reason Simbad returned more than one {prefix} entry for {self.input_ID}.')
                else:
                    dicts[i]['ID'] = tempid[0]

        return self.simbad['entry']
        
    def query_KIC(self, radius = 10.0*u.arcsec):
        from astroquery.vizier import Vizier
        time.sleep(0.1)
        
        
        kicjob = Vizier.query_object(object_name = self.KIC['ID'], catalog = 'V/133/kic', radius = radius)
        
        self.KIC['entry'] = kicjob
        return self.KIC['ID'], self.KIC['entry']
    
    def query_TIC(self, radius = 10.0*u.arcsec):
        from astroquery.mast import Catalogs
        time.sleep(0.1)
        
        
        ticjob = Catalogs.query_object(objectname=self.TIC['ID'], catalog='TIC', objType='STAR', radius = radius)
        
        self.TIC['entry'] = ticjob
        return self.TIC['ID'], self.TIC['entry']
    
    def query_GaiaDR2(self):
        if not self.GDR2['ID']:
            
            if 'Gaia DR2' in self.input_ID:
                self.GDR2['ID'] = self.input_ID
            else:
                print(f'To query the Gaia DR2 catalog, {self.input_ID} must be a valid Gaia source id.')
                print('Try using the archives.query_simbad() method.')            
        
        if self.GDR2['ID']:
            from astroquery.gaia import Gaia
            time.sleep(0.1)

            gid = int(self.GDR2['ID'].replace('Gaia DR2 ', ''))
            adql_query = "select * from gaiadr2.gaia_source where source_id=%i" % (gid)

            job = Gaia.launch_job(adql_query).get_results()
            idx = np.where(job['source_id'].quantity == gid)[0][0]
            self.GDR2['entry'] = job[idx]

        return self.GDR2['ID'], self.GDR2['entry']
    


In [129]:
arch = archives('KIC8006161')

In [130]:
simbad = arch.query_simbad()

In [131]:
arch.query_GaiaDR2();

In [133]:
g = arch.GDR2['entry']

In [134]:
g['bp_rp']

0.98118925

In [72]:
gid = int(arch.gaia['ID'].replace('Gaia DR2 ', ''))
adql_query = "select * from gaiadr2.gaia_source where source_id=%i" % (gid)

In [74]:
from astroquery.gaia import Gaia

gaiajob = Gaia.launch_job(adql_query)

In [86]:
res = gaiajob.get_results()

In [89]:
idx = res['source_id'].quantity == gid

In [ ]:
cats = ['KIC', 'TIC', 'EPIC', 'Gaia DR1', 'Gaia DR2', 'HD', 'HIP', 'HR', 'TYC']

In [ ]:
access_archives('HD16417')

if 'KIC' in ID
 format anf add to dict
capitalized
kiclike
ticlike
epiclike
if Gaia DR2 in ID
 format and add to dict

Simbad first

loop through dict and access MAST, TIC, KIC, Simbad, Gaia Archive

HD, HIP, HR

In [ ]:
Q.to_pandas()

In [ ]:
IDs = str(Q['IDS'][0])

In [ ]:
idx = IDs.index('Gaia DR2')

In [ ]:
IDs = IDs[idx:].replace("'", '').replace('Gaia DR2','').strip(' ')

In [ ]:
IDs

In [ ]:
from astroquery.gaia import Gaia
job = Gaia.launch_job("select * from gaiadr2.gaia_source where source_id=%i" % (int(IDs)))

In [ ]:
tab = job.get_results()#.to_pandas()

In [ ]:
int(tab['source_id'].quantity) #== int(IDs)

In [23]:
a = '/home/nielsemb/work/bla.txt'
b = '/home/nielsemb/work/'
import os

In [24]:
os.path.join(*[b,a])

'/home/nielsemb/work/bla.txt'

In [44]:
'{0}'.format(r'bla','bli')

'bla'

In [46]:
import numpy as np
np.log(10.0)

2.302585092994046

In [71]:
class star():
    def __call__(self):
        print('Star has been called')
    def __init__(self):
        self.b = 0
        self.c = self
    

In [72]:
a = star()

In [73]:
vars(a)

{'b': 0, 'c': <__main__.star at 0x7f44fc1b8b70>}

In [74]:
a.c()

Star has been called


In [75]:
hasattr(a, 'c')

True

In [78]:
np.arange(-50,0)

array([-50, -49, -48, -47, -46, -45, -44, -43, -42, -41, -40, -39, -38,
       -37, -36, -35, -34, -33, -32, -31, -30, -29, -28, -27, -26, -25,
       -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12,
       -11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1])